In [1]:
!git clone https://github.com/yizhou-wang/cruw-devkit.git

fatal: destination path 'cruw-devkit' already exists and is not an empty directory.


In [2]:
%cd cruw-devkit

/home/vr-lab/Asish/8th Sem Radar Project/Codes/cruw-devkit


In [3]:
from cruw import CRUW
import numpy as np
import matplotlib.pyplot as plt
from cruw.mapping import ra2idx, idx2ra
import math
import os
import random

In [4]:
data_root='/mnt/disk1/CRUW/ROD2021'
dataset = CRUW(data_root=data_root, sensor_config_name='sensor_config_rod2021')


In [5]:
scene_list = os.listdir("/home/vr-lab/Asish/8th Sem Radar Project/Datasets/CRUW/TRAIN_RAD_H/")
scene_list.sort()
print(scene_list)

['2019_04_09_BMS1000', '2019_04_09_BMS1001', '2019_04_09_BMS1002', '2019_04_09_CMS1002', '2019_04_09_PMS1000', '2019_04_09_PMS1001', '2019_04_09_PMS2000', '2019_04_09_PMS3000', '2019_04_30_MLMS000', '2019_04_30_MLMS001', '2019_04_30_MLMS002', '2019_04_30_PBMS002', '2019_04_30_PBMS003', '2019_04_30_PCMS001', '2019_04_30_PM2S003', '2019_04_30_PM2S004', '2019_05_09_BM1S008', '2019_05_09_CM1S004', '2019_05_09_MLMS003', '2019_05_09_PBMS004', '2019_05_09_PCMS002', '2019_05_23_PM1S012', '2019_05_23_PM1S013', '2019_05_23_PM1S014', '2019_05_23_PM1S015', '2019_05_23_PM2S011', '2019_05_29_BCMS000', '2019_05_29_BM1S016', '2019_05_29_BM1S017', '2019_05_29_MLMS006', '2019_05_29_PBMS007', '2019_05_29_PCMS005', '2019_05_29_PM2S015', '2019_05_29_PM3S000', '2019_09_29_ONRD001', '2019_09_29_ONRD002', '2019_09_29_ONRD005', '2019_09_29_ONRD006', '2019_09_29_ONRD011', '2019_09_29_ONRD013']


In [11]:
for scene in scene_list:
    anno_path = os.path.join('/home/vr-lab/Asish/8th Sem Radar Project/Datasets/CRUW/TRAIN_RAD_H_ANNO',scene+'.txt')
    
    with open(anno_path) as f:
        lines = f.readlines()
    
    c = 0
    crd_mat = []
    for line in lines:
        words = line.split(" ")
        seq_num = int(words[0])
        rng = float(words[1])
        azm = float(words[2])
        label = words[3][:-1]

        img_name = os.path.join('/home/vr-lab/Asish/8th Sem Radar Project/Datasets/CRUW/TRAIN_RAD_H',scene,'RADAR_RA_H/%06d_0000.npy' % seq_num)
        fimg = np.load(img_name)
        fimg = np.sqrt(np.square(fimg[:,:,0])+np.square(fimg[:,:,1]))
        imgw, imgh = fimg.shape
        
        pimg = np.zeros((imgw+10, imgh+50))
        pimg[5:-5,25:-25] = fimg

                
        if c != seq_num:
            n_points = 4
            xz = np.random.randint(0, imgw, n_points)
            yz = np.random.randint(0, imgh, n_points)
            points  = [[xz[i], yz[i]] for i in range(n_points)]
#             print(points)
            for i in crd_mat:
                xp = i[0]
                yp = i[1]
                for j in points:
                    xz = j[0]
                    yz = j[1]
                    if xp-5<=xz<=xp+5 and yp-25<=yz<=yp+25:
#                         print(xz,",", yz)
                        points.remove([xz,yz])
#             print("Valid points : ", points)
            
            kseq_num = seq_num-1
            kimg_name = os.path.join('/home/vr-lab/Asish/8th Sem Radar Project/Datasets/CRUW/TRAIN_RAD_H',scene,'RADAR_RA_H/%06d_0000.npy' %kseq_num)
            kimg = np.load(kimg_name)
            kimg = np.sqrt(np.square(kimg[:,:,0])+np.square(kimg[:,:,1]))
            
            bimg = np.zeros((imgw+10, imgh+50))
            bimg[5:-5,25:-25] = kimg
#             plt.imshow(bimg)
#             Since we are cchanging the image dimensions, the points will also change
            
            num = 0
            for j in points:
                xz = j[0] + 5
                yz = j[1] + 25
                seg_bimg = bimg[xz-5:xz+5,yz-25:yz+25]
                path1 = os.path.join('/home/vr-lab/Asish/8th Sem Radar Project/Datasets/CRUW_seg/none', scene +"_"+str(num)+"_%06d_0000.npy" % seq_num)
                np.save(path1,seg_fimg)
                num = num+1
            crd_mat = []

        rid, aid = ra2idx(rng, azm, dataset.range_grid, dataset.angle_grid)

        path2 = os.path.join('/home/vr-lab/Asish/8th Sem Radar Project/Datasets/CRUW_seg', label, scene + "_%06d_0000.npy" % seq_num)
        seg_fimg = pimg[rid+5-5:rid+5+5,aid+25-25:aid+25+25]
        
#         plt.imshow(seg_fimg)

        np.save(path2,seg_fimg)
        c = seq_num
        crd_mat.append([rid,aid])
#         print(seq_num," : ",crd_mat," : ",label)
    print(scene," over")


2019_04_09_BMS1000  over
2019_04_09_BMS1001  over
2019_04_09_BMS1002  over
2019_04_09_CMS1002  over
2019_04_09_PMS1000  over
2019_04_09_PMS1001  over
2019_04_09_PMS2000  over
2019_04_09_PMS3000  over
2019_04_30_MLMS000  over
2019_04_30_MLMS001  over
2019_04_30_MLMS002  over
2019_04_30_PBMS002  over
2019_04_30_PBMS003  over
2019_04_30_PCMS001  over
2019_04_30_PM2S003  over
2019_04_30_PM2S004  over
2019_05_09_BM1S008  over
2019_05_09_CM1S004  over
2019_05_09_MLMS003  over
2019_05_09_PBMS004  over
2019_05_09_PCMS002  over
2019_05_23_PM1S012  over
2019_05_23_PM1S013  over
2019_05_23_PM1S014  over
2019_05_23_PM1S015  over
2019_05_23_PM2S011  over
2019_05_29_BCMS000  over
2019_05_29_BM1S016  over
2019_05_29_BM1S017  over
2019_05_29_MLMS006  over
2019_05_29_PBMS007  over
2019_05_29_PCMS005  over
2019_05_29_PM2S015  over
2019_05_29_PM3S000  over
2019_09_29_ONRD001  over
2019_09_29_ONRD002  over
2019_09_29_ONRD005  over
2019_09_29_ONRD006  over
2019_09_29_ONRD011  over
2019_09_29_ONRD013  over


In [ ]:
plt.imshow(kimg)

In [ ]:
plt.imshow(fimg)

In [ ]:
id, aid = ra2idx(10.0369, 0.5760, dataset.range_grid, dataset.angle_grid)

In [ ]:
img = np.load('/home/vr-lab/Asish/8th Sem Radar Project/Datasets/CRUW_SR/2019_05_09_BM1S008/000100_0000.npy')
print(img.shape)

In [ ]:
num=10
print(rid,aid)
plt.imshow(img[rid-num:rid+num,aid-num:aid+num])

In [ ]:
a = np.load("/home/vr-lab/Asish/8th Sem Radar Project/Datasets/CRUW_seg/pedestrian/2019_05_29_PM2S015_000313_0000.npy")

In [ ]:
plt.imshow(a)

In [ ]:
a = np.random.randint(0, 256, 10).reshape((5, 2))
print(a)
n_sampled_points = 5
a[np.random.randint(0, a.shape[0], n_sampled_points)]

In [ ]:
imgw, imgh = 10, 20
n_points = 20
x = np.random.randint(0,imgw,n_points)
y = np.random.randint(0,imgh,n_points)
points  = [[x[i], y[i]] for i in range(len(x))]
print(points)